In [3]:
# Import all stuff we need
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.pylab as pylab

from brian import Network, Equations, NeuronGroup, Connection,\
    SpikeMonitor, raster_plot, StateMonitor, clear, reinit
from brian.stdunits import ms, mV

%matplotlib inline
pylab.rcParams['figure.figsize'] = 12, 8  # changes figure size (width, height) for larger images

In [4]:
clear(True, True) 
reinit()# To reinit BRIAN clocks and remove all old BRIAN objects from namespace,
# it's usually a good idea to put this at the beginning of a script

In [9]:
# The equations defining our neuron model
eqs_string = ''' dV/dt = 1.0/tau * (mu - V) + I_syn : volt
             mu : volt
             tau : ms
             I_syn : volt/ms
             '''

In [10]:
# Our model parameters
tau_e = 15*ms # membrane time constant (for excitatory synapses)
tau_1 = 10*ms # membrane time constant (for inhibitory synapses)
tau_syn_e = 3*ms # exc synaptic time constant tau2 in paper
tau_syn_i = 2*ms # inh synaptic time constant tau2 in paper
syn_delay = 1*ms # synaptic delay between two neurons tau1 in paper
V_th = -50*mV # firing threshold
V_reset = -65*mV # reset potential
refr_period = 5*ms # absolute refractory period

# Our parameters for the bias mu
mu_e = np.array([1.1, 1.2]) # bias for excitatory neurons
mu_i = np.array([1.0, 1.05]) # bias for inhibitory neurons

# Number of neurons
N_e = 4000 # number of exc neurons
N_i = 1000 # number of inh neurons

# Connection probability
conn_prob_ee = 0.2
conn_prob_ii = 0.5
conn_prob_ei = 0.5
conn_prob_ie = 0.5

# Duration of our simulation
duration = 2000*ms

In [11]:
# Let's create an equation object from our string and parameters
model_eqs = Equations(eqs_string)

In [12]:
# Let's create 5000 neurons
all_neurons = NeuronGroup(N=N_e+N_i,
                          model=model_eqs,
                          threshold=V_th,
                          reset=V_reset,
                          refractory=refr_period,
                          freeze = True,
                          method='Euler',
                          compile=True)

In [13]:
# Divide the neurons into excitatory and inhibitory ones
neurons_e = all_neurons[0:N_e]
neurons_i = all_neurons[N_e:N_e+N_i]